In [7]:
from pydap.client import open_url
from datetime import datetime
import numpy as np
import pandas as pd
import time
from datetime import timedelta
import pyproj
import xarray as xr
from scipy.interpolate import LinearNDInterpolator
import matplotlib.pyplot as plt
import math
import pickle
from datetime import date
import os
import plotly.express as px
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib
import cartopy as cart
from tqdm import tqdm

In [10]:
def fetch_oscar_data(start_day, end_day):
    for i in tqdm(range(start_day, end_day+1)):
        res = fetch_oscar_one_day(i)
        if res:
            u_list = []
            v_list = []
            lat_list = []
            lon_list = []
            for index, lat in enumerate(res[0]):
                for index2, lon in enumerate(res[1]):
                    u_list.append(res[2][index][index2])
                    v_list.append(res[3][index][index2])
                    lat_list.append(lat)
                    lon_list.append(lon)
            df = pd.DataFrame()
            df['u'] = u_list
            df['v'] = v_list
            df['lat'] = lat_list
            df['lon'] = lon_list
            df['time'] = np.zeros(len(df['lon']))
            df = df.assign(time=i*24)
            df = df.loc[df['lon'] < 380 ]
            df['lon'] = df['lon']%360
            df = df.loc[df['lat'] <= 38 ]
            df = df.loc[df['lat'] >= 38 ]
            df.dropna(inplace = True)
            df.to_csv("oskar_data/" + str(i) + ".csv" ,index=False)
        
def fetch_oscar_one_day(day):
    try:
        url = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/oscar/preview/L4/oscar_third_deg/oscar_vel' + str(day) + '.nc.gz?latitude[0:1:480],longitude[0:1:1200],u[0:1:0][0:1:0][0:1:480][0:1:1200],v[0:1:0][0:1:0][0:1:480][0:1:1200]'       
        dataset = open_url(url, output_grid=False)
        lats = np.array(dataset.latitude[:])
        lons = np.array(dataset.longitude[:])
        u = np.squeeze(np.array(dataset.u[:][:][:][:]))            
        v = np.squeeze(np.array(dataset.v[:][:][:][:]))         
        return lats, lons, u, v
    except:
        return None

In [11]:
fetch_oscar_data(10000, 10024)

100%|██████████| 25/25 [01:34<00:00,  3.77s/it]


In [13]:
df = pd.read_csv("oskar_data/10004.csv")
min(df['lat'])

-78.33333333333333